In [ ]:
'''

用戶菜單功能介紹

    用戶能透過點擊菜單，進行我方希冀之業務功能。
    
流程
    準備菜單的圖面設定檔
    讀取安全設定檔上的參數
    將菜單設定檔傳給Line
    對Line上傳菜單照片
    檢視現有的菜單
    將菜單與用戶做綁定
    將菜單與用戶解除綁定
    刪除菜單

'''

In [1]:
'''
菜單設定檔

    設定圖面大小、按鍵名與功能
    
'''

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "Trans Menu",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 21,
        "y": 17,
        "width": 1217,
        "height": 804
      },
      "action": {
        "type": "postback",
        "text": "^to_english",
        "data": "^to_english"
      }
    },
    {
      "bounds": {
        "x": 1279,
        "y": 17,
        "width": 1209,
        "height": 804
      },
      "action": {
        "type": "postback",
        "text": "^to_japanese",
        "data": "^to_japanese"
      }
    },
    {
      "bounds": {
        "x": 19,
        "y": 857,
        "width": 1217,
        "height": 804
      },
      "action": {
        "type": "postback",
        "text": "^to_french",
        "data": "^to_french"
      }
    },
    {
      "bounds": {
        "x": 1270,
        "y": 854,
        "width": 1217,
        "height": 804
      },
      "action": {
        "type": "uri",
        "uri": "https://www.merriam-webster.com/"
      }
    }
  ]
}
"""

In [3]:
'''

讀取安全檔案內的字串，以供後續程式碼調用

'''
from pprint import pprint
import json
secretFileContentJson=json.load(open("../line_secret_key",'r'))
pprint(secretFileContentJson)
print("="*20)


channel_access_token = secretFileContentJson.get("channel_access_token")
secret_key = secretFileContentJson.get("channel_access_token")
self_user_id = secretFileContentJson.get("self_user_id")

print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))


{'channel_access_token': 'L06yAhHo+DPBW/2eWk0cMdoonf26tYryXMDMJvOjazT5Sua4jxlee9OvgzyGchIy6A1VRPu3EdA6CPRAm16n0GZu89ejJNobwyy2dmMwH5z8EdqZM++fDsk3820KBGto4aAkhgugNzRKyC9XfcL0awdB04t89/1O/w1cDnyilFU=',
 'notify_token': 'A*********************************2sUq',
 'rich_menu_id': 'richmenu-80865577578e0b99d092d341730c18b8',
 'secret_key': 'e0f9898e0ee49ab21eb3b2f901b2d0c4',
 'self_user_id': 'U60d8a876f6837d1af4cac912adb1ee02',
 'server_url': '194666cb.ngrok.io'}
L06yAhHo+DPBW/2eWk0cMdoonf26tYryXMDMJvOjazT5Sua4jxlee9OvgzyGchIy6A1VRPu3EdA6CPRAm16n0GZu89ejJNobwyy2dmMwH5z8EdqZM++fDsk3820KBGto4aAkhgugNzRKyC9XfcL0awdB04t89/1O/w1cDnyilFU=
e0f9898e0ee49ab21eb3b2f901b2d0c4
U60d8a876f6837d1af4cac912adb1ee02


In [4]:
'''

將先前準備的菜單設定檔，以Post消息寄發給Line

    設定Line的遠端位置
    設定消息的基本安全憑證
    寄發消息，並取得回應之Id
    

'''

import requests

# 載入設定檔
menuJson=json.loads(menuRawData)

# 上傳到 Line
createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'

# 將認證碼 channel_access_token 放到封包的表頭上
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

#print(createMenuRequestHeader)

# 傳給 Line
lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

# 印出結果
print(lineCreateMenuResponse)
print(lineCreateMenuResponse.text)


<Response [200]>
{"richMenuId":"richmenu-1d31ef96fa5a8fa78a0a33720f1f4ebd"}


In [6]:
'''

將先前準備的菜單照片，以Post消息寄發給Line
    取得上面設定檔的Id
    設定Line的遠端位置
    設定消息的基本安全憑證
    上傳照片，並取得回傳成果

'''

# 取得功能選單的 id
uploadRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")
print(uploadRichMenuId)
#'https://api.line.me/v2/bot/richmenu/{richMenuId}/content'

# 設定上傳位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 為封包設定認證的公司章 (channel_access_token)
uploadMenuRequestHeader={'Content-Type':'image/jpeg',
                         'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 讀取圖片
uploadImageFile=open("../res/MyTranslator_menu.png",'rb')

# 上傳圖片
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,
                                     headers=uploadMenuRequestHeader,
                                     data=uploadImageFile)

# 觀看結果
print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-1d31ef96fa5a8fa78a0a33720f1f4ebd
https://api.line.me/v2/bot/richmenu/richmenu-1d31ef96fa5a8fa78a0a33720f1f4ebd/content
<Response [200]>
{}


In [7]:
'''

將選單綁定到特定用戶身上
    取出上面得到的菜單Id及用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# https://api.line.me/v2/bot/user/{userId}/richmenu/{richMenuId}

# 取得菜單的 id
linkRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")

#將 user_id 和 richmenu_id 組合成一個網址
linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (secretFileContentJson["self_user_id"], linkRichMenuId)
print(linkMenuEndpoint)

# 為封包設定安全標籤(蓋公司章)
linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 將需求傳給 Line
lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
print(lineLinkMenuResponse)
print(lineLinkMenuResponse.text)

https://api.line.me/v2/bot/user/U60d8a876f6837d1af4cac912adb1ee02/richmenu/richmenu-1d31ef96fa5a8fa78a0a33720f1f4ebd
<Response [200]>
{}


In [8]:
'''

檢視用戶目前所綁定的菜單
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

#  https://api.line.me/v2/bot/user/{userId}/richmenu

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)


https://api.line.me/v2/bot/user/U60d8a876f6837d1af4cac912adb1ee02/richmenu
<Response [200]>
{"richMenuId":"richmenu-1d31ef96fa5a8fa78a0a33720f1f4ebd"}


In [ ]:
'''

解除選單與特定用戶的綁定
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUnregisterUserMenuResponse=requests.delete(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)

In [9]:
'''

檢視帳號內，有哪些選單
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''
from pprint import pprint
# 設定Line的遠端位置
listMenuEndpoint="https://api.line.me/v2/bot/richmenu/list"

# 設定消息的基本安全憑證
listMenuHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}    

# 發送消息告知
lineLisrMenuResponse = requests.get(listMenuEndpoint,headers=listMenuHeader)
pprint(lineLisrMenuResponse.text)

('{"richmenus":[{"richMenuId":"richmenu-1d31ef96fa5a8fa78a0a33720f1f4ebd","name":"Trans '
 'Menu","size":{"width":2500,"height":1686},"chatBarText":"查看更多資訊","selected":true,"areas":[{"bounds":{"x":21,"y":17,"width":1217,"height":804},"action":{"type":"postback","data":"^to_english","text":"^to_english"}},{"bounds":{"x":1279,"y":17,"width":1209,"height":804},"action":{"type":"postback","data":"^to_japanese","text":"^to_japanese"}},{"bounds":{"x":19,"y":857,"width":1217,"height":804},"action":{"type":"postback","data":"^to_french","text":"^to_french"}},{"bounds":{"x":1270,"y":854,"width":1217,"height":804},"action":{"type":"uri","uri":"https://www.merriam-webster.com/"}}]},{"richMenuId":"richmenu-3050c89fcd6c69673b450c38163e3466","name":"translatorMenu","size":{"width":2500,"height":1686},"chatBarText":"transMenu","selected":true,"areas":[{"bounds":{"x":7,"y":9,"width":864,"height":820},"action":{"type":"message","text":"[::text:]本次交易"}},{"bounds":{"x":876,"y":0,"width":786,"height":830}